# Determine the drag coefficient with DRIA

To construct the orbit arcs we are going to use the `arc_times.txt` file.
- just use the orbit start and stop times
- turn on the DRIA usage: ` 'cd_model'       : {'input': 'DRIA'},
`

### TO DO:
- Add the autoselection for the month G2B and initial conditions file
- Determine what the `cd_type` input var is doing
- go get the CD plots...
- determine precise definition of sailboat and airplane mode
- add the pickle thing to make the files go faster




## Read arc_times

In [1]:
import pandas as pd

arc_timesfile = '/data/SatDragModelValidation/data/inputs/raw_inputdata/data_ICESat2/arc_times.txt'


### Make a function that finds the right range of dates in this file.
arcs = pd.read_csv(arc_timesfile, 
            sep = ',',
#             dtype=object,
            names = [
                'arc'         ,
                'epoch_start' ,
                'epoch_stop'  ,
                'orbit_start' ,
                'orbit_stop'  ,
                    ],)


arcs_yyyyddd = [x.strip() for x in arcs['arc'].values.tolist()]
epochstart   = [x.strip() for x in arcs['orbit_start'].values.tolist()]
epochstop    = [x.strip() for x in arcs['orbit_stop' ].values.tolist()]


Select the dates within the month of interest

In [2]:
# arcs_yyyyddd
# epochstart



In [3]:
from pygeodyn.PYGEODYN import Pygeodyn
import datetime


run_list = [
#                 'msis2',
#                 'tiegcm_oc',
#                 'ctipe_oc',
#                 'jb2008',
                'dtm2020_o',
#                 'gitm',
#                 'hasdm_oc',
           ]

dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    if i =='msis2':
        run_dict[i]={}
        run_dict[i]['num'] = 5
        run_dict[i]['model_path'] = None
    if i =='dtm2020_o':
        run_dict[i]={}
        run_dict[i]['num'] = 3
        run_dict[i]['model_path'] = None
    if i =='jb2008':
        run_dict[i]={}
        run_dict[i]['num'] = 1
        run_dict[i]['model_path'] = None
        
print(run_dict)

{'dtm2020_o': {'num': 3, 'model_path': None}}


In [4]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os

# file_raw_ICs = f"/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"\
#           +"ICESat2_RawEphem_2018_10.txt"

g2b_path = "/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"
dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'icesat2/O2R2023_longimeperiod/1_DRIAruns/'

# month_list = [, 'nov', 'dec']
month_list = ['oct']

obj = {}

for imonth,month in enumerate(month_list):
    if month=='oct':
        m_num = 10
    if month=='nov':
        m_num = 11
    if month=='dec':
        m_num = 12

    file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_2018_{m_num}.txt"
    file_g2b     = f"pce_icesat2_pso_2018_{m_num}"
    
    input_arcs       = []
    input_epochstart = []
    input_epochstop  = []

    for i,val in enumerate(arcs_yyyyddd):
        if pd.to_datetime(val[:8],format="%Y.%j").month == m_num:
    #         print(val, i)
            input_arcs.append(      arcs_yyyyddd[i])
            input_epochstart.append(epochstart[i])
            input_epochstop.append( epochstop[i])
            print(f"{i} {arcs_yyyyddd[i]:10}  , {epochstart[i]:20},  {epochstop[i]:20}")

    for i,den in enumerate(run_list):

        settings_icesat2= {# Basic input settings
                     'satellite'      : {'input': 'icesat2'},
                     'den_model'      : {'input': den},
                     'run_type'       : {'input': 'DataReduction_PCE'},
                     'run_specifier'  : {'input': '_'+month},
                     'cd_model'       : {'input': 'DRIA'},
    #                  'cd_model'       : {'input': 'BWDRAG'},
                     'file_string'    : {'input': 'test'},
                     'model_data_path' : {'input': run_dict[den]['model_path']},
                     # Force Model settings
                      'cd_type'               : {'input':"Fixed_CD"},
                      'cd_value'              : {'input':2.500000},
                      'scaling_factor'        : {'input':False},
    #                   'hours_between_cd_adj'  : {'input':24},
                      #### Comment for unadjusted run:
                      'cd_adjustment_boolean' : {'input':False },
            #### DRIA CD Model Parameters
            'cd_model_params' : {'input':{ 
                        'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
                        'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
                           ###  Alpha should be some value from 0 to 1
                        'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient
                        'KL'     : '0.0D0'    ,  #!  langmuir parameter
                        'FRACOX' : '1.0D0'   ,  #!  fraction of surface covered by atomic oxygen
                       }},

                      #### ---------------------------------------
                     # Run
                      'arc_type'       : {'input':'Nominal30hr_and_AB'},      
                      'step'           : {'input': 10.},
                      'orbfil_step'    : {'input': 120.},
                      'which_ICfile'   : {'input':file_raw_ICs},
                      'which_g2bfile'  : {'input':file_g2b},
                        #
                      'arc'            : {'input':input_arcs},
                      'epoch_start'    : {'input': input_epochstart},
                      'epoch_stop'     : {'input':input_epochstop},  
                       #                                
                      'global_options' : {'input':'pso_2018'},
                     # Request read on raw outputs
                      'request_data'   : {'input': ['Trajectory_orbfil', 
                                                   'Density', 
                                                   'Residuals_summary',
                                                   'DragFile',
                                                   'AdjustedParams'
                                                   ]},
                  #end dict
                  }




        sat = Pygeodyn(settings_icesat2, use_file=False)
        sat.run_arcs()
    #     obj[den] =  sat.getData_BigData_lowmemory()
    #     obj[den] = vars(obj[den])
        gc_collect()


#     pickleName = f'_{month}_DRIAtest.pkl'

#     pickle_file = dir_save+den+pickleName
#     if not os.path.exists(pickle_file):
#         print('Must create pickle file...')
#         print('   ',  pickle_file)
#         print('   ', 'Reading Geodyn Data')

#         ### Load the data into an object
#         sat = Pygeodyn(settings_icesat2, use_file=False)
#         obj = sat.getData_BigData_lowmemory()
#         gc_collect()

#         #### Pickle the object to save it
#         print('   ', 'Saving pickle')
#         filehandler = open(pickle_file, 'wb') 
#         pickle.dump(vars(obj), filehandler)
#         filehandler.close()
#         obj = 0
#         print('   ', 'Saved pickle')

# obj = {}
# for i,model in enumerate(run_list):
#     for imonth,month in enumerate(month_list):
#         pickleName = f'_{month}_DRIAtest.pkl'

#         ### Load the data if the pickles exist
#         print()
#         print()
#         gc_collect()

#         pickle_file = dir_save+model+pickleName

#         filehandler = open(pickle_file, 'rb') 
#         obj[month+model] = pickle.load(filehandler)
#         filehandler.close()
#         print('Loaded data from pickle... ',  model)
    
    
# ## -----------------------------------------------------
# ## startdate = "2018-10-14"
# ## enddate   = "2018-11-02"  
# ## sat = Pygeodyn(settings_icesat2, use_file=False)
# ## sat.initialize_timeperiod_stage1(startdate, enddate,
# ##                                  overwrite_exat=False, 
# ##                                  overwrite_ICtext=False)
# ## import sys
# ## sys.exit(0)

0 2018.287    , 2018-10-13 21:18:00 ,  2018-10-15 02:42:00 
1 2018.288    , 2018-10-14 21:18:00 ,  2018-10-16 02:42:00 
2 2018.289A   , 2018-10-15 21:18:00 ,  2018-10-16 23:00:00 
3 2018.290A   , 2018-10-17 00:24:00 ,  2018-10-18 02:42:00 
4 2018.291    , 2018-10-17 21:18:00 ,  2018-10-19 02:42:00 
5 2018.292    , 2018-10-18 21:18:00 ,  2018-10-20 02:42:00 
6 2018.293    , 2018-10-19 21:18:00 ,  2018-10-21 02:42:00 
7 2018.294    , 2018-10-20 21:18:00 ,  2018-10-22 02:42:00 
8 2018.295    , 2018-10-21 21:18:00 ,  2018-10-23 02:42:00 
9 2018.296    , 2018-10-22 21:18:00 ,  2018-10-24 02:42:00 
10 2018.297    , 2018-10-23 21:18:00 ,  2018-10-25 02:42:00 
11 2018.298    , 2018-10-24 21:18:00 ,  2018-10-26 02:42:00 
12 2018.299    , 2018-10-25 21:18:00 ,  2018-10-27 02:42:00 
13 2018.300    , 2018-10-26 21:18:00 ,  2018-10-28 02:42:00 
14 2018.301A   , 2018-10-27 21:18:00 ,  2018-10-28 16:00:00 
15 2018.301B   , 2018-10-28 17:36:00 ,  2018-10-29 02:42:00 
16 2018.302A   , 2018-10-28 21:18:

KeyboardInterrupt: 

In [ ]:
# file_raw_ICs
# file_g2b

#### Plot Settings:


In [ ]:
from pygeodyn.pygeodyn_plot_scalingfactors import *

coldict = {}
coldict['msis2']     = "#2ca02c"  # 'tab:green'
coldict['dtm2020_o'] = "#d62728"  # 'tab:red'
coldict['jb2008']    = "orange"   
coldict['hasdm_oc']  = "#1f77b4"     


## Plot Drag State

In [ ]:
run_dict

In [ ]:
%load_ext autoreload
%autoreload 2
from pygeodyn.pygeodyn_plot_scalingfactors import *

models_dens   = { }
scale_cadence =  0
for model in run_dict.keys():
    
    for imonth,month in enumerate(month_list):
        print('---Getting continuous densities for ', month+model)

        models_dens[month+model] = get_continuous_densities(obj, month+model, scale_cadence)



In [ ]:
# models_dens

In [ ]:
# obj['jb2008']['DragFile']['2018.303']

In [ ]:
# arc = obj['jb2008']['global_params']['arc_input'][0]

# print(pd.to_datetime(arc, format = '%Y.%j'))
# print(pd.to_datetime(arc, format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm'))


In [ ]:
%load_ext autoreload
%autoreload 2


def PLOT__SATDRAG_STATE(fig, obj_m1, den_dict):
    
    SHOW_alldata = True
    
    model_m1 = obj_m1['global_params']['prms']['den_model']
    if model!='hasdm_oc':
        opac_val = 1
    else:
        opac_val = 1

    for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
        epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
        hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
        frachours =(hrs/24)
        #
        if len(arc) == 9:
            maneuv_indicator = arc[8]
        else:
            maneuv_indicator = ''
        arc_type = obj_m1['global_params']['prms']['arc_type']
        if arc_type == "Nominal30hr_and_AB":
            arc_name =arc[:8]+ maneuv_indicator
        else:
            arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        



        drag_data        = obj_m1['DragFile'][arc_name]
        drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
        den_df           = obj_m1['Density' ][arc_name]
        
#         print(arc[:8])
        ### Cut off the extra time for the regular days
        start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
        end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

#         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
        ### SELECT ONLY THE MIDDLE 24-HOURS
        drag_data = drag_data.query( \
                f"{start_arc.year}"         \
               +f"{start_arc.month:02d}"    \
               +f"{start_arc.day:02d}"      \
               +f"{start_arc.hour:02d}"     \
               +f"{start_arc.minute:02d}"   \
               +f"{start_arc.second:02d}"   \
               +f" <= Date < "                     \
               +f"{end_arc.year}"       \
               +f"{end_arc.month:02d}"  \
               +f"{end_arc.day:02d}"    \
               +f"{end_arc.hour:02d}"   \
               +f"{end_arc.minute:02d}" \
               +f"{end_arc.second:02d}" \
            )
        ### SELECT ONLY THE MIDDLE 24-HOURS
        den_df = den_df.query( \
                f"{start_arc.year}"         \
               +f"{start_arc.month:02d}"    \
               +f"{start_arc.day:02d}"      \
               +f"{start_arc.hour:02d}"     \
               +f"{start_arc.minute:02d}"   \
               +f"{start_arc.second:02d}"   \
               +f" <= Date < "                     \
               +f"{end_arc.year}"       \
               +f"{end_arc.month:02d}"  \
               +f"{end_arc.day:02d}"    \
               +f"{end_arc.hour:02d}"   \
               +f"{end_arc.minute:02d}" \
               +f"{end_arc.second:02d}" \
            )


        indplot=3

#         date_rms.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) + datetime.timedelta(int(arc.split('.')[1]))- datetime.timedelta(hours=12) ))
#         rms_totals.append(obj_m1['Statistics'][arc]['T_RMS'].values[0])

#         iters = obj_m1['run_parameters'+arc_name]['total_iterations']
#         for itime in obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD']:
#             date_scalefactor.append(itime)
#             ScalingFactor.append(obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD'][itime]['CURRENT_VALUE'])


        if SHOW_alldata:
            fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
                                       y=drag_data['CD'][::indplot],
                                     mode='markers',
                                     opacity=0.5,
                                     marker=dict(color= coldict[model_m1],size=2,),
                                     showlegend=False,),
                                     secondary_y=False,
                                     row=2, col=1,
                                 )
        (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    
        fig.add_trace(go.Scatter(x=time_avg,
                                 y=avg_cd,
                           mode='lines',
                           opacity=1,
                               line = dict(dash ='solid', color = coldict[model_m1], width=3),
                           showlegend=False), row=2, col=1)

        if SHOW_alldata:
            fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
                                       y=drag_data['TOTAREA'][::indplot],
                                     mode='markers',
                                     opacity=0.5,
                                     marker=dict(color= coldict[model_m1],size=2,),
                                     showlegend=False,),
                                     secondary_y=False,
                                     row=3, col=1,
                                 )
        (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    
        fig.add_trace(go.Scatter(x=time_avg,
                                 y=avg_area,
                           mode='lines',
                           opacity=1,
                               line = dict(dash ='solid', color = coldict[model_m1], width=3),
                           showlegend=False), row=3, col=1)

        if SHOW_alldata:
            fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
                                       y=drag_data['CD'][::indplot]*drag_data['TOTAREA'][::indplot],
                                     mode='markers',
                                     opacity=0.5,
                                     marker=dict(color= coldict[model_m1],size=2,),
                                     showlegend=False,),
                                     secondary_y=False,
                                     row=4, col=1,
                                 )
        fig.add_trace(go.Scatter(x=time_avg,
                                 y=avg_cd*avg_area,
                           mode='lines',
                           opacity=1,
                               line = dict(dash ='solid', color = coldict[model_m1], width=3),
                           showlegend=False), row=4, col=1)

        
        if SHOW_alldata:
            fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
                                       y=drag_data['VELREL'][::indplot],
#                                          name= plot_name+arc,
                                     mode='markers',
                                     opacity=0.5,
                                     marker=dict(color= coldict[model_m1],size=2,),
                                     line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
                                     showlegend=False, ),
                                     secondary_y=False,
                                     row=5, col=1,
                                 )
        (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['VELREL'],drag_data['Lat'])    
        fig.add_trace(go.Scatter(x=time_avg,
                                 y=avg,
                           mode='lines',
                           opacity=1,
                               line = dict(dash ='solid', color = coldict[model_m1], width=3),
                           showlegend=False), row=5, col=1)

        if SHOW_alldata:
            fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
                                   y=drag_data['SpeedRatio'][::indplot],
#                                          name= plot_name+arc,
                                     mode='markers',
                                     opacity=0.5,
                                     marker=dict(color= coldict[model_m1],size=2,),
                                     line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
                                     showlegend=False, ),
                                     secondary_y=False,
                                     row=6, col=1,
                                 )
        (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['SpeedRatio'],drag_data['Lat'])    
        fig.add_trace(go.Scatter(x=time_avg,
                                 y=avg,
                           mode='lines',
                           opacity=1,
                               line = dict(dash ='solid', color = coldict[model_m1], width=3),
                           showlegend=False), row=6, col=1)
        if SHOW_alldata:
            fig.add_trace(go.Scatter(x=den_df['Date'][::indplot],
                                 y=den_df['rho (kg/m**3)'][::indplot],
                       ### name= model_m1,
                       mode='markers',
                       opacity=0.5,
                           marker=dict(color=coldict[model_m1],size=2),
                           line = dict(dash ='solid', color = coldict[model_m1], width=2),
                       showlegend=False), row=1, col=1)
        
        (time_avg, d_avg ) = orbit_avg_generic(den_df['Date'],den_df['rho (kg/m**3)'],
                                               den_df['Lat'])    
        fig.add_trace(go.Scatter(x=time_avg,
                                 y=d_avg,
                           mode='lines',
                           opacity=1,
                               marker=dict(color=coldict[model_m1],size=1),
                               line = dict(dash ='solid', color = coldict[model_m1], width=3),
                           showlegend=False), row=1, col=1)

    

    
    
#     (time_avg, d_avg ) = orbit_avg_generic(den_dict['dates'],
#                                    den_dict['den'],
#                                    den_dict['lat'])
#     ### -----------------------------------------------------------------------------------------------------
#     ###     Orbit Averaged Density
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=d_avg,
#                                ### name= model_m1,
#                                mode='markers',
#                                opacity=opac_val,
#                                    marker=dict(color= coldict[model_m1],size=2),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=1, col=1)
    
    
#                 time_avg,CD_avg, area_avg_rolling, CD_std = orb_avg_param(obj_m1.__dict__['DragFile'], arc_string, 'TOTAREA')
#                 fig.add_trace(go.Scattergl(x=time_avg,
#                                            y=area_avg_rolling,
#     #                                          name= plot_name+arc,
#                                              mode='markers+lines',
#                                              opacity=1,
#                                              marker=dict(color=col,size=2,),
#                                              line = dict(shape='hvh', dash ='solid', color = col, width=3),
#                                              showlegend=False, ),
#                                              secondary_y=False,
#                                              row=3, col=1,
#                                          )



        

        
    return(fig)







fig  = make_subplots(
    rows=6, cols=1,
#     subplot_titles=(['Density (kg/m^3)'   , 
# #                      'CD*A'               ,
#                      'CD'                 ,   
#                      'Relative Velocity'  , 
#                      'Speed Ratio',
#                     ]),
    vertical_spacing = 0.05,
#     horizontal_spacing = 0.05,
    shared_xaxes=True)



# run_dict = { }
# rms_total_return = {}

for model in run_dict.keys():
    for imonth,month in enumerate(month_list):

#     for ialph,alpha in enumerate(alphas):   
         # fig,rms_total_return = PLOT__intrack_residuals_w_rms_w_cd(fig, Obj_Geodyn['msis2'] , 0,  arc_listlist)
#         run_dict[val+"alpha_"+alpha] = ialph

        fig = PLOT__SATDRAG_STATE(fig, obj[month+model],   models_dens[month+model])

    
#######################################################
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = True
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)

    
    fig.update_yaxes(title_text="Density", 
                     type="log", 
                     exponentformat= 'power',row=1, col=1)
    fig.update_yaxes(title_text="CD", 
                     exponentformat= 'power',row=2, col=1)
    fig.update_yaxes(title_text="Total Area", 
                     exponentformat= 'power',row=3, col=1)
    fig.update_yaxes(title_text="CD*Area", 
                     exponentformat= 'power',row=4, col=1)

    fig.update_yaxes(title_text="Rel. Velocity [m/s] ", 
                     exponentformat= 'power',row=5, col=1)
    fig.update_yaxes(title_text="Speed Ratio", 
                     exponentformat= 'power',row=6, col=1)

    ###
    ###  DATE on Final x-Axis only
#     fig.update_xaxes(title="Date",
#                      row=4, col=1)
#     fig.update_xaxes(title="Date",
#                      row=5, col=2)




fig.update_layout(
#                   title = '',
                  autosize=False,    width=1000,    height=1100,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title



# fig.add_vrect(x0="2018-10-16 23:00:00" ,
#               x1="2018-10-17 00:24:00" ,
#                 fillcolor="red",
#                 opacity=0.25,
#                 line_width=0)
# fig.add_vrect(x0="2018-10-28 16:00:00" ,
#               x1="2018-10-28 17:36:00" ,
#                 fillcolor="red",
#                 opacity=0.25,
#                 line_width=0)
# fig.add_vrect(x0="2018-10-29 10:18:00" ,
#               x1="2018-10-29 11:42:00" ,
#                 fillcolor="red",
#                 opacity=0.25,
#                 line_width=0)
# fig.add_vrect(x0="" ,
#               x1="" ,
#                 fillcolor="red",
#                 opacity=0.25,
#                 line_width=0)
# fig.add_vrect(x0="" ,
#               x1="" ,
#                 fillcolor="red",
#                 opacity=0.25,
#                 line_width=0)

fig.show(config=config)




In [ ]:
den_df['Date']

In [ ]:
for i,val in enumerate(input_arcs):
    print(f"{i:3} {input_arcs[i]:10}| {pd.to_datetime(input_arcs[i][:8],format='%Y.%j').strftime('%b %d')}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}")